# Live plots of Time Tagger measurements in a jupyter notebook

This jupyter notebook gives an example how to show live plots of Time Tagger Measurements. As an example, we enable the internal test signal and continuously plot the countrate.

### Idea

We use **plotly** for the plotting as it integrates nicely with the javascript side of jupyter and has better live plotting capabilities than matplotlib.

We use **asyncio** for background plot updates for its ease of use.

### Installation

Install __[anaconda python](https://www.anaconda.com/distribution/)__

Install jupyter, plotly, nodejs

```
conda install jupyter
conda install plotly (version>=5.20.0)
conda install nodejs
conda install ipywidgets (version>=8.0.0)
```

If you want to use jupyter lab, you may need to install jupyterlab extensions for plotly following the README on __[https://github.com/plotly/plotly.py](https://github.com/plotly/plotly.py)__


In [ ]:
from TimeTagger import *

import pkg_resources
import asyncio
import numpy as np
if (pkg_resources.require("ipywidgets>=8.0.0") and pkg_resources.require("plotly>=5.20.0")):
    from ipywidgets import Button
    import plotly.graph_objs as go

Connect to the TimeTagger and enable the internal test signal on channel 1

In [ ]:
# connect to the TimeTagger
tagger = createTimeTagger()

# enable the internal test signal on channels 
tagger.setTestSignal(1, True)

Create a plot and mechanisms to update it in the background

In [ ]:
# create a counter with 100 ms binwidth and one bin and 
trace = Counter(tagger, [1], 10**9, 1000)

# create a figure widget and a plot
fig_trace = go.FigureWidget()
fig_trace.add_scatter(x=trace.getIndex(), y=trace.getData()[0])

async def update_trace():
    """Update the plot every 0.1 s"""
    while True:
        fig_trace.data[0].y = trace.getData()[0]
        await asyncio.sleep(0.1)

# If this cell is re-excecuted and there was a previous task, stop it first to avoid a dead daemon
try:
    task_trace.cancel()
except:
    pass
loop = asyncio.get_event_loop()
task_trace = loop.create_task(update_trace())
    
# create a stop button
button_trace = Button(description='stop')
button_trace.on_click(lambda a: task_trace.cancel())

display(fig_trace,button_trace)


You can stop the background plotting either by clicking on the stop button above or by executing `task_trace.cancel()`

In the next example we produce a bar plot by using a countrate with only one bin.

In [ ]:
# create a counter with 100 ms binwidth and one bin and 
point = Counter(tagger, [1], 10**11, 1)

# create a figure widget and a plot
fig_bar = go.FigureWidget()
fig_bar.add_bar(x=point.getIndex(), y=point.getData()[0])
fig_bar.layout.yaxis={'range':[0,100000]}

async def update_bar():
    """Update the plot every 0.1 s"""
    while True:
        fig_bar.data[0].y = point.getData()[0]
        await asyncio.sleep(0.1)

# If this cell is re-excecuted and there was a previous task, stop it first to avoid a dead daemon
try:
    task_bar.cancel()
except:
    pass
loop = asyncio.get_event_loop()
task_bar = loop.create_task(update_bar())

# create a stop button
button_bar = Button(description='stop')
button_bar.on_click(lambda a: task_bar.cancel())

display(fig_bar, button_bar)
